In [9]:
# O modulo time aqui foi utilizado para esperar o carregamento das paginas atraves do firefox
# o modulo webdriver e necessario para definir qual navegador sera utilizado para fazer a automacao
# o modulo Select sera utilizado para interagir com a caixa de selecao onde sera definido o ano em que quero buscar os dados
# esse modulo sera utilizado para 

import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup 

In [289]:
#chave_pesq = "itaipava"
#chave_pesq = "budweiser"
#chave_pesq = "refrigerante"
chave_pesq = "bolo"



A linha abaixo define qual e o navegador que queremos utilizar, lembrando que eu instalei somente o driver para conexao com o firefox, mas existem tambem para Chrome e InternetExplorer
Abaixo foi definido qual e o site que quero acessar

In [290]:
driver = webdriver.Chrome()
driver.get("https://buscapreco.sefaz.am.gov.br/home")

### Fazer recapcha manualmente antes de rodar o próximo

In [291]:
descricao = driver.find_element_by_id("descricaoProd")
pesquisar = driver.find_element_by_xpath("/html/body/main/div/form/div[2]/div[3]/button")
descricao.send_keys(chave_pesq)
pesquisar.click()

In [292]:
paginacao = driver.find_element_by_class_name("pagination")
html = paginacao.get_attribute("innerHTML")
soup = BeautifulSoup(html,"html.parser")

In [293]:
def qtde_pag():
    lista = driver.find_element_by_class_name("pagination")
    teste1 = lista.get_attribute("innerHTML")
    teste2 = BeautifulSoup(teste1,"html.parser")        
    link = teste2.findAll('a')
    ultimo = link[len(link)-1]['href']
    return int(ultimo.split('/')[4])

### conteiner dos small cards

In [294]:
def busca_pagina(url_pag):    
    #print(url_pag)
    driver.get(url_pag)
    pagina = driver.find_element_by_xpath("/html/body/main/div[3]/div[1]").get_attribute("innerHTML")
    pagina_soup = BeautifulSoup(pagina,"html.parser")
    return pagina_soup

### Pega as infomacao de um small card

In [295]:
def to_float(preco):
    return float(preco.replace('R$ ','').replace(',','.'))

In [296]:
def get_info_small(small_card):
    info = small_card.findAll('b')
    nome = info[0].text
    preco = to_float(info[1].text)
    estabelecimento = info[2].text
    endereco = small_card.findAll('span')[1].text
    return [nome, preco, estabelecimento,endereco]

### Pega as informações de todos os small card de uma página

In [297]:
def get_info_page(small_cards):
    info_page = []
    for small_card in small_cards:
        info_page.append(get_info_small(small_card))
    return info_page
    

In [298]:
data_all = []
print(qtde_pag())
url_base = 'https://buscapreco.sefaz.am.gov.br/item/grupo/page/'
for a in range(qtde_pag()):
    url_pag = '{}{}'.format(url_base,a + 1)
    print(url_pag)
    driver.get(url_pag)
    soup = busca_pagina(url_pag)
    small_cards = soup.findAll('div',attrs={'class':'card small p hoverable'})
    data_all += (get_info_page(small_cards))
    
    #time.sleep(5)
    
for dados in data_all:
    print(dados)

#a = 0
#url_pag = 'https://buscapreco.sefaz.am.gov.br/item/grupo/page/{}'.format(a + 1)
#soup = busca_pagina(url_pag)

73
https://buscapreco.sefaz.am.gov.br/item/grupo/page/1
https://buscapreco.sefaz.am.gov.br/item/grupo/page/2
https://buscapreco.sefaz.am.gov.br/item/grupo/page/3
https://buscapreco.sefaz.am.gov.br/item/grupo/page/4
https://buscapreco.sefaz.am.gov.br/item/grupo/page/5
https://buscapreco.sefaz.am.gov.br/item/grupo/page/6
https://buscapreco.sefaz.am.gov.br/item/grupo/page/7
https://buscapreco.sefaz.am.gov.br/item/grupo/page/8
https://buscapreco.sefaz.am.gov.br/item/grupo/page/9
https://buscapreco.sefaz.am.gov.br/item/grupo/page/10
https://buscapreco.sefaz.am.gov.br/item/grupo/page/11
https://buscapreco.sefaz.am.gov.br/item/grupo/page/12
https://buscapreco.sefaz.am.gov.br/item/grupo/page/13
https://buscapreco.sefaz.am.gov.br/item/grupo/page/14
https://buscapreco.sefaz.am.gov.br/item/grupo/page/15
https://buscapreco.sefaz.am.gov.br/item/grupo/page/16
https://buscapreco.sefaz.am.gov.br/item/grupo/page/17
https://buscapreco.sefaz.am.gov.br/item/grupo/page/18
https://buscapreco.sefaz.am.gov.br

In [299]:

def gravar_csv(data_all):
    with open("{}.csv".format(chave_pesq),"w") as f:
        #headers = 'nome;link;ano;mes'
        #s = "".join(headers)
        #f.write('{}\n'.format(s))
        for l in data_all:
            f.write('{};{};{};{}\n'.format(l[0],l[1],l[2],l[3]))
            
gravar_csv(data_all)

In [301]:
import csv
def gravar_csv2(data_all):
    with open('teste.csv', 'w') as csvfile:
        fieldnames = ['nome', 'preco', 'estabelecimento', 'endereco']
        writer = csv.DictWriter(csvfile, delimiter=';', fieldnames=fieldnames)
        #writer.writeheader()    
        for l in data_all:
            writer.writerow({'nome': l[0],'preco': l[1],'estabelecimento': l[2],'endereco': l[3],})
gravar_csv2(data_all)

In [302]:
driver.close()